<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math
import sys
import os

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))

if module_path not in sys.path:
    sys.path.append(module_path)

import utils
    
from rf.regression import multiple_forests as multiple

from utils.reader.densitymap import load_directory
from tqdm import tqdm_notebook, tqdm
from ipywidgets import *

%load_ext autoreload
%autoreload 1
%aimport utils.reader.densitymap, rf.regression
%matplotlib notebook

In [2]:
# setup
simulation = 'check/csvs/obstacle/'
experiment = 'check/csvs/experiment/'

number_of_targets = 3

In [12]:
# load training (simulation data)
train = load_directory(simulation, number_of_targets, file_filter=lambda x: 'reduced' in x)

# load test (experiment data)
test = load_directory(experiment, number_of_targets, file_filter=lambda x: 'filtered' in x)

Loading directory check/csvs/experiment/: 100%|██████████| 7/7 [00:01<00:00,  5.58it/s]


In [ ]:
display(train[1])

In [15]:
# run
regressors, \
normed_prediction, \
(score_training, score_test, score_oob), \
errors = multiple( \
    {'samples': train[0], 'targets': train[1]}, \
    {'samples': test[0], 'targets': test[1]}, \
    number_of_trees=50)

Learning targets: 100%|██████████| 3/3 [02:21<00:00, 36.22s/it]


In [16]:
display(errors)

{'euklid': {'mean': 0.75328100592693,
  'std': 0.27087667570762064,
  'percent': 53.265010742995614},
 'mean_abs_error': array([0.45018219, 0.19713929, 0.49654198]),
 'mean_error': array([-0.45018219, -0.04635979,  0.49654198]),
 'rmse': array([0.54289278, 0.26949628, 0.52292029])}

In [21]:
from utils.reader.densitymap import ingredient as density_ingredient, load_directory
from sacred import Experiment
from sacred.observers import FileStorageObserver

%aimport utils.reader.densitymap

ex = Experiment('my_experiment', ingredients=[density_ingredient], interactive=True)

ex.observers.append(FileStorageObserver.create('check/runs'))

@ex.main
def main(_run, _log):
    _log.info(_run.config)
    # load training (simulation data)
    train = load_directory(simulation, number_of_targets, file_filter=lambda x: 'reduced' in x)

    # load test (experiment data)
    test = load_directory(experiment, number_of_targets, file_filter=lambda x: 'filtered' in x)
    
result = ex.run()

INFO - my_experiment - Running command 'main'
INFO - my_experiment - Started run with ID "2"
INFO - main - {'seed': 662360225, 'reader': {'densitymap': {'seperator': ';', 'file_filter': <function <lambda> at 0x7f69a4b09378>}}}
Loading directory check/csvs/experiment/: 100%|██████████| 7/7 [00:01<00:00,  5.63it/s]
INFO - my_experiment - Completed after 0:00:03


In [6]:
print(result.info)

{}
